In [ ]:
import hvplot.polars  # noqa
import hydra
import numpy as np
import pandas as pd
import polars as pl
import rootutils
import torch
from rich import print

torch.set_float32_matmul_precision("high")
rootutils.setup_root("../", pythonpath=True)

In [ ]:
# setting up paths
from pathlib import Path

from dotenv import dotenv_values

paths = dotenv_values()

In [ ]:
run_dir = Path(paths["RUN_DIR"])
run_dir

In [ ]:
from src.data.vcc_datamodule import VCCDataModule
from src.models.vcc_lightning import VCCModule

In [ ]:
with hydra.initialize("../config", version_base=None):
    conf = hydra.compose("train", ["model=model_attention"])

In [ ]:
print(conf)

In [ ]:
datamodule = hydra.utils.instantiate(conf.data.datamodule)
datamodule

In [ ]:
net = hydra.utils.instantiate(conf.model.net)
model = VCCModule.load_from_checkpoint(run_dir / "VCC_02_08_2025/18-40/last.ckpt", net=net)

In [ ]:
model.eval()

In [ ]:
trainer = hydra.utils.instantiate(conf.trainer)

In [ ]:
prediction = trainer.predict(model, datamodule)

In [ ]:
len(prediction)

# Trying out cell-eval

In [ ]:
# Save data so you dont have to do previous steps again and again
X = np.vstack(prediction).astype(np.float32)
print(X.shape)
np.savez("../results/preds.npz", X)

In [ ]:
X = np.load("../results/preds.npz")["arr_0"]  # Care
X

In [ ]:
data_path = Path(paths["DATA_DIR"])

In [ ]:
validation_data = pl.read_csv(data_path / "processed-data/pert_counts_Validation.csv")
validation_data

In [ ]:
gene_names = pl.scan_parquet(conf.data.datamodule.control_data_path).collect_schema().names()[1:]
gene_names[:10]

In [ ]:
pred_df = pl.DataFrame(X, schema=gene_names)
pred_df

In [ ]:
pred_df.hvplot.hist(y=["SAMD11", "CCNL2"], bins=50)

In [ ]:
with open("../results/gene_names.csv", "w") as f:
    for gene in gene_names:
        f.write(f"{gene},\n")

In [ ]:
target_genes = (
    validation_data.with_columns(target_gene_2=pl.col("target_gene").repeat_by("n_cells"))
    .select(pl.col("target_gene_2").explode().alias("target_gene"))
    .to_pandas()
)  # Generating the row annotations

In [ ]:
# Processing non-targeting data

nt_data = pl.read_parquet(conf.data.datamodule.control_data_path)
nt_target_genes = (
    nt_data.with_columns(pl.lit("non-targeting").alias("target_gene"))
    .select("target_gene")
    .to_pandas()
)
nt_data = nt_data.select(pl.all().exclude("sample_index")).to_numpy().astype(np.float32)

In [ ]:
X = np.vstack([X.astype(np.float32), nt_data])
target_genes = pd.concat([target_genes, nt_target_genes])

target_genes

In [ ]:
import anndata as ad

In [ ]:
result = ad.AnnData(
    X=X.astype(np.float32),  # So scipy.sparse doesn't cry
    var=pd.DataFrame(index=gene_names),
    obs=target_genes,
)
result

In [ ]:
result.write_h5ad("../results/prediction_020825.h5ad")